# Библиотеки

In [18]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage.exposure import rescale_intensity
from scipy import ndimage
import numpy as np
import cv2
import csv

# Загрузка и обработка данных

In [2]:
core_dir = 'C:\\Users\\horch\\Desktop\\Data mining\\Kaggle Amazon Rainforest\\'
train_dir = core_dir + 'train-jpg\\'
test_dir = core_dir + 'test-jpg\\'
save_dir = core_dir+ 'save-jpg\\'

cathegories = ['agriculture', 'artisinal_mine', 'bare_ground', 
                      'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 
                      'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 
                      'road', 'selective_logging', 'slash_burn', 'water']

train_data = pd.read_csv(core_dir + 'train.csv')
train_data.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


# Функции

подготовка обучающей выборки по имени категории
принимает категорию, возвращает массив из 0 и 1 (1 - файл содержит категорию, 0 - иначе)

In [3]:
def TrainingSetByCathegory(cathegory):
    df = pd.DataFrame()
    
    df['image_name'] = train_data.image_name.values
    df['indicator'] = np.zeros(train_data.shape[0])
    
    df.loc[df['image_name'].isin(cathegory_dict[cathegory]), ['indicator']] = 1
    
    return df.indicator.values

функция предназначенная для конвертации .jpg изображения в numpy массив
если transparency = False, то каждая точка представляется 3 числами иначе - 4

In [4]:
def ImageToNumpy(img_name, img_dir = train_dir, img_type = '.jpg', transparency = False):
    img = Image.open(train_dir + img_name + img_type)
    #img.load()
    data = np.asarray( img, dtype="int32" )
    
    if transparency:
        return data
    else:
        return data[:, :, 0:3]

In [15]:
# функция свертки (различные ядра)
def ConvImg(img_name, img_dir = train_dir, img_type = '.jpg'):
    # различные ядра свертки
    # construct average blurring kernels used to smooth an image
    smallBlur = np.ones((7, 7), dtype="float") * (1.0 / (7 * 7))
    largeBlur = np.ones((21, 21), dtype="float") * (1.0 / (21 * 21))
 
    # construct a sharpening filter
    sharpen = np.array((
        [0, -1, 0],
        [-1, 5, -1],
        [0, -1, 0]), dtype="int")

    # construct the Laplacian kernel used to detect edge-like
    # regions of an image
    laplacian = np.array((
        [0, 1, 0],
        [1, -4, 1],
        [0, 1, 0]), dtype="int")
 
    # construct the Sobel x-axis kernel
    sobelX = np.array((
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]), dtype="int")
 
    # construct the Sobel y-axis kernel
    sobelY = np.array((
        [-1, -2, -1],
        [0, 0, 0],
        [1, 2, 1]), dtype="int")
    
    
    image = cv2.imread(train_dir + img_name + img_type)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    conv_img = ndimage.convolve(gray,laplacian)
    return conv_img

array([[  5,   2,   0, ...,   5, 251, 243],
       [  2,   1,   1, ...,  12,   5, 252],
       [  1,   2,   2, ...,  15,   6,   4],
       ..., 
       [  4,   1,   0, ...,   3, 254, 249],
       [  3,   1, 254, ..., 255, 252, 247],
       [  4,   3, 253, ..., 255, 252, 251]], dtype=uint8)

# Вычисления

создадим словарь категорий
каждой категории будет соответствовать список файлов, в которых данная категория присутствует

In [6]:
cathegory_dict = {cathegory: [] for cathegory in cathegories}

ind = 0
for tag in train_data.tags.values:
    file_name = train_data.image_name[ind]
    
    cathegory_names_list = tag.split(' ')
    
    for cathegory in cathegory_names_list:
        cathegory_dict[cathegory].append(file_name)
    
    ind += 1   

In [17]:
to_csv = []
for img in train_data.image_name:
    to_csv.append((img,ConvImg(img)))    

[('train_0', array([[251,   0,   1, ..., 253, 252,   1],
         [255,   0,   4, ..., 251,   0,   1],
         [  2,   7,   0, ...,   0,   1,   0],
         ..., 
         [  0,   2, 254, ..., 249, 255,   2],
         [  0, 255, 255, ..., 250, 253,   4],
         [255,   1,   4, ..., 251, 253,   7]], dtype=uint8)),
 ('train_1', array([[  5,   2,   0, ...,   5, 251, 243],
         [  2,   1,   1, ...,  12,   5, 252],
         [  1,   2,   2, ...,  15,   6,   4],
         ..., 
         [  4,   1,   0, ...,   3, 254, 249],
         [  3,   1, 254, ..., 255, 252, 247],
         [  4,   3, 253, ..., 255, 252, 251]], dtype=uint8)),
 ('train_2', array([[  5, 252, 254, ...,   0,   1,   2],
         [  3,   0, 252, ..., 255,   0,   3],
         [  4, 251, 253, ..., 249,   1,   6],
         ..., 
         [  3,   4, 254, ...,   2,   0, 251],
         [  1,   3,   3, ...,   2, 252, 250],
         [  2,   5,   1, ...,   3, 255, 253]], dtype=uint8)),
 ('train_3', array([[  2,   1, 255, ..., 249, 

In [19]:
with open('G:\GIT\Amazon\laplacian_conv.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for item in to_csv:
        csv_writer.writerow([item])